In [ ]:
import os
import cv2
import time, gc
import numpy as np
import pandas as pd
from math import ceil
import seaborn as sns
from glob import glob
from tqdm.auto import tqdm
from tqdm import tqdm_notebook as tqdm
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.graph_objects as go

from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda
from keras.models  import Sequential
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPool2D,Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import efficientnet.keras as efn

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models

In [ ]:
spark = SparkSession.builder.appName('ReadData').getOrCreate()
sc = spark.sparkContext

In [ ]:
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

In [ ]:
# Read data_purchase.csv file
train_df_ = spark.read.csv(("train.csv", inferSchema=True, header=True)
test_df_ = spark.read.csv('test.csv', inferSchema=True, header=True)
class_map_df = spark.read.csv('class_map.csv', inferSchema=True, header=True)
sample_sub_df = spark.read.csv(('sample_submission.csv', inferSchema=True, header=True)

In [ ]:
train_df.printSchema()

In [ ]:
train_df.show(5)

# EDA using PySpark

In [ ]:
# purely for heuristic exercise to show date manipulation can be done
# date column was not actually used for training
df1 = train_df.withColumn('train_dt', unix_timestamp('tran_dt', 'yyyy-MM-dd').cast('timestamp'))
df1 = df1.filter("train_dt like 'PST%' or tain_dt like 'CST%' or train_dt like 'EST%' or train_dt like 'MST%'")

In [ ]:
# PRIMARY DATA WRANGLING OPERATION HERE
# join with another table, group by some column, summarize (count), arrange rows by some column, and displaying the first 10 rows (^=
counts = train_df.groupby('grapheme_root').count()
train_df.join(counts, 'grapheme_root').select('grapheme_root','grapheme','count').orderBy(['count'], ascending=[0]).show(10)
# can also do by filter(.count > 1000) - choose some threshold level 

In [ ]:
# need to join back with the original table
df2 = train_df.join(counts, 'grapheme_root').select('grapheme_root','grapheme','count').orderBy(['count'], ascending=[0])
# Bar chart - Top 10 events based on total purchased tickets amount
df2.plot(kind= 'bar', x='grapheme_counts', y='count', figsize=(20,7), grid=True, rot=30, ylim=(4000, 75000))

In [ ]:
# clean data - remove whitespace from any text data
charReplace = udf(lambda x: x.replace(u' ',''))
df1 = df.withColumn('grapheme_root',charReplace('grapheme_root'))
df1.show(3)

In [ ]:
# Drop columns
df2 = df1.drop('id')
df2.count()

In [ ]:
# remove the duplicate rows based on class_maping
class_map_df = class_map_df.drop_duplicates(subset=['grapheme'])

In [ ]:
# histogram in probabilsitic distribution
fig = px.histogram(train_df, x="grapheme_root", histnorm='probability', title="Grapheme Occurance Count in order of Grapheme Root")
fig.show()

In [ ]:
# vowel diacritic occurrence count
fig = px.histogram(train_df, x="vowel_diacritic", title="Vowel Diacritic Occurance Count")
fig.show()

In [ ]:
# vowel diacritic count
dfb = train_df.groupby("vowel_diacritic").count()
fig = px.bar(dfb["grapheme"].sort_values(), y="grapheme", title="Vowel Diacritic Occurance Count Sorted from Low to High")
fig.show()

In [ ]:
# consonant diacritic count
fig = px.histogram(train_df, x="consonant_diacritic", title="Vowel Diacritic Occurance Count")
fig.show()

In [ ]:
# Grapheme Count: 
dfb = df.groupby("consonant_diacritic").count()
fig = px.bar(dfb["grapheme"].sort_values(), y="grapheme", title="Consonant Diacritic Occurance Count Sorted from Low to High")
fig.show()

In [ ]:
HEIGHT = 137
WIDTH = 236

In [ ]:
train_df = train_df.toPandas()

# EDA Using Pandas ^^

In [ ]:
# in order to show the actual images we use pd.read_parquet function
def load_as_npa(file):
    df = pd.read_parquet(file)
    return df.iloc[:, 0], df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH)

def image_from_char(char):
    image = Image.new('RGB', (WIDTH, HEIGHT))
    draw = ImageDraw.Draw(image)
    myfont = ImageFont.truetype('/kaggle/input/bengaliai/hind_siliguri_normal_500.ttf', 120)
    w, h = draw.textsize(char, font=myfont)
    draw.text(((WIDTH - w) / 2,(HEIGHT - h) / 2), char, font=myfont)

    return image

In [ ]:
image_ids0, images0 = load_as_npa('train_image_data_0.parquet')

In [ ]:
f, ax = plt.subplots(5, 5, figsize=(16, 8))
ax = ax.flatten()

for i in range(25):
    ax[i].imshow(images0[i], cmap='Greys')

In [ ]:
print("Number of unique grapheme_root: {}".format(train_df['grapheme_root'].nunique()))

In [ ]:
fig = go.Figure(data=[go.Histogram(x=train_df['grapheme_root'])])
fig.update_layout(title_text='`grapheme_root` values')
fig.show()

In [ ]:
# Most common grapheme_root values
x = train_df['grapheme_root'].value_counts().sort_values()[-20:].index
y = train_df['grapheme_root'].value_counts().sort_values()[-20:].values
fig = go.Figure(data=[go.Bar(x=x, y=y)])
fig.update_layout(title_text='Most common grapheme_root values')
fig.show()

In [ ]:
# Least common grapheme_root values
x = train_df['grapheme_root'].value_counts().sort_values()[:20].index
y = train_df['grapheme_root'].value_counts().sort_values()[:20].values
fig = go.Figure(data=[go.Bar(x=x, y=y)])
fig.update_layout(title_text='Least common grapheme_root values')
fig.show()

In [ ]:
# vowel_diacritic values
x = train_df['vowel_diacritic'].value_counts().sort_values().index
y = train_df['vowel_diacritic'].value_counts().sort_values().values
fig = go.Figure(data=[go.Bar(x=x, y=y)])
fig.update_layout(title_text='vowel_diacritic values')
fig.show()

In [ ]:
# consonant_diacritic values
x = train_df['consonant_diacritic'].value_counts().sort_values().index
y = train_df['consonant_diacritic'].value_counts().sort_values().values
fig = go.Figure(data=[go.Bar(x=x, y=y)])
fig.update_layout(title_text='consonant_diacritic values')
fig.show()

In [ ]:
# this function outputs the top/bottom n rows in a nice format with data types
def get_n(df, field, n, top=True):
    top_graphemes = df.groupby([field]).size().reset_index(name='counts')['counts'].sort_values(ascending=not top)[:n]
    top_grapheme_roots = top_graphemes.index
    top_grapheme_counts = top_graphemes.values
    top_graphemes = class_map_df[class_map_df['component_type'] == field].reset_index().iloc[top_grapheme_roots]
    top_graphemes.drop(['component_type', 'label'], axis=1, inplace=True)
    top_graphemes.loc[:, 'count'] = top_grapheme_counts
    return top_graphemes

In [ ]:
# get the top 10 grapheme roots
top_10_roots = get_n(train_df, 'grapheme_root', 10)
top_10_roots

In [ ]:
# get the bottom 5 grapheme roots
bottom_10_roots = get_n(train_df, 'grapheme_root', 10, False)
bottom_10_roots

In [ ]:
# get the top 5 vowels
top_5_vowels = get_n(train_df, 'vowel_diacritic', 5)
top_5_vowels

In [ ]:
# get the top 5 consonants
top_5_consonants = get_n(train_df, 'consonant_diacritic', 5)
top_5_consonants

# using PySpark to run machine learning 

## logistic regression and random forest

In [ ]:
IMG_SIZE = 64

In [ ]:
inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))

In [ ]:
# merge two tables - all the training parquet images to the training set 
for i in range(4):
    train_df_ = pd.merge(pd.read_parquet(f'/Users/jihunlee/Desktop/bengali/train_image_data_{i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)

In [ ]:
# drop unneeded columns
X_train = train_df_.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
X_train.shape

In [ ]:
# resize the images before running machine learning - later deep learning algorithms
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=64
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [ ]:
X_train.shape
X_train = resize(X_train2)/255
X_train.shape

In [ ]:
# for simplicity, only output grapheme_root
indexer1 = StringIndexer(inputCols=X_train.columns[-1], outputCol="grapheme_root").setHandleInvalid("skip")
inputs = [indexer1.getOutputCol()]
encoder = OneHotEncoderEstimator(inputCols=inputs,  \
                                 outputCols=["grapheme_root"])

#run it through a pipeline
pipeline = Pipeline(stages=[indexer1, encoder])
pipeline = pipeline.fit(X_train).transform(X_train)

In [ ]:
#gather feature vector and identify features
assembler = VectorAssembler(inputCols = inputs,
                            outputCol = 'grapheme_root')
pipeline = assembler.transform(pipeline)

In [ ]:
%%time
from pyspark.ml.classification import LogisticRegression

# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol='label')

# Fit the model to the data.
lgrm = lgr.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test_df)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

In [ ]:
%%time
from pyspark.ml.classification import RandomForestClassifier

# Set parameters for the Random Forest.
rfc = RandomForestClassifier(maxDepth=5, numTrees=15, impurity="gini", labelCol="label", predictionCol="prediction")

# Fit the model to the data.
rfcm = rfc.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
predictions = rfcm.transform(test_df)

In [ ]:
predictions2 = rfcm.transform(test_df)

In [ ]:
print(evaluator.evaluate(predictions2, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions2, {evaluator.metricName: "f1"}))

# Using Sci-kit Learn and Keras ^___^ to Train 1) James-40 2) a

In [ ]:
# MY 40 LAYER MODEL

In [ ]:
# 25 convolution layer with doubling filters
# SAME padding
# drop rate = 30%
# batch normalization = 15%

In [ ]:
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)

head_root = Dense(168, activation = 'softmax')(dense)
head_vowel = Dense(11, activation = 'softmax')(dense)
head_consonant = Dense(7, activation = 'softmax')(dense)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ideally I should use cross validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)

In [ ]:
X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(X_train.shape)

Y_train_root = pd.get_dummies(train_df_['grapheme_root']).values
Y_train_vowel = pd.get_dummies(train_df_['vowel_diacritic']).values
Y_train_consonant = pd.get_dummies(train_df_['consonant_diacritic']).values

print(f'Training images: {X_train.shape}')
print(f'Training labels root: {Y_train_root.shape}')
print(f'Training labels vowel: {Y_train_vowel.shape}')
print(f'Training labels consonants: {Y_train_consonant.shape}')

In [ ]:
# this data augmentation function doesn't quite work
# ideally I should implement this before running deep learning models

In [ ]:
from tensorflow import keras
class ImageDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)

In [ ]:
classifier = []
batch_size = 256
epochs = 5 # only 5 epochs for computational ease


histories = []
history = model.fit_generator(datagen.flow(x_train, {'dense_3': y_train_root, 'dense_4': y_train_vowel, 'dense_5': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size )
 
histories.append(history)

In [ ]:
# custom make performance function per epochs
%matplotlib inline
def plot_loss(cls,epoch,title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0,epoch),cls.history['loss'],label = 'train_loss')
    plt.plot(np.arange(0,epoch),cls.history['dense_3_loss'],label = 'train_root_loss')
    plt.plot(np.arange(0,epoch),cls.history['dense_4_loss'],label = 'train_vowel_loss')
    plt.plot(np.arange(0,epoch),cls.history['dense_5_loss'],label = 'train_consonant_loss')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_3_loss'],label = 'val_train_root_loss')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_4_loss'],label = 'val_train_vowel_loss')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_5_loss'],label = 'val_train_consonant_loss')
    plt.title(title)
    plt.legend(loc='upper right')
    plt.xlabel('Number of Epoch ')
    plt.ylabel('Loss')
    plt.show()
    
    
    
def plot_accuracy(cls,epoch,title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0,epoch),cls.history['dense_3_accuracy'],label = 'train_root_accuracy')
    plt.plot(np.arange(0,epoch),cls.history['dense_4_accuracy'],label = 'train_vowel_accuracy')
    plt.plot(np.arange(0,epoch),cls.history['dense_5_accuracy'],label = 'train_comnsonant_accuracy')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_3_accuracy'],label = 'val_train_root_accuracy')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_4_accuracy'],label = 'val_train_vowel_accuracy')
    plt.plot(np.arange(0,epoch),cls.history['val_dense_5_accuracy'],label = 'val_train_consonant_accuracy')
    plt.title(title)
    plt.xlabel('Number of Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
for dataset in range(1):
    plot_loss(histories[dataset], epochs, f'Training Dataset: {dataset}')
    plot_accuracy(histories[dataset], epochs, f'Training Dataset: {dataset}')

In [ ]:
# this time use multi-classification
pred_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)

    X_test = resize(df_test_img, need_progress_bar=False)/255
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(pred_dict):
        pred_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(pred_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample.head()

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AvgPool2d(2)
        # vowel_diacritic
        self.fc1 = nn.Linear(512,11)
        # grapheme_root
        self.fc2 = nn.Linear(512,168)
        # consonant_diacritic
        self.fc3 = nn.Linear(512,7)
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return x1,x2,x3

In [ ]:
    def __init__(self):
        super(ResNet34,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AvgPool2d(2)
        # vowel_diacritic
        self.fc1 = nn.Linear(512,11)
        # grapheme_root
        self.fc2 = nn.Linear(512,168)
        # consonant_diacritic
        self.fc3 = nn.Linear(512,7)
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return x1,x2,x3

In [ ]:
model = ResNet34().to(device)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=0.05)
criterion = nn.CrossEntropyLoss()
batch_size=32

In [ ]:
epochs = 5
model.train()
losses = []
accs = []
for epoch in range(epochs):
    reduced_index =train.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']).apply(lambda x: x.sample(5)).image_id.values
    reduced_train = train.loc[train.image_id.isin(reduced_index)]
    reduced_data = data_full.loc[data_full.image_id.isin(reduced_index)]
    train_image = GraphemeDataset(reduced_data,reduced_train)
    train_loader = torch.utils.data.DataLoader(train_image,batch_size=batch_size,shuffle=True)
    
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    for idx, (inputs,labels1,labels2,labels3) in tqdm(enumerate(train_loader),total=len(train_loader)):
        inputs = inputs.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        
        optimizer.zero_grad()
        outputs1,outputs2,outputs3 = model(inputs.unsqueeze(1).float())
        loss1 = criterion(outputs1,labels1)
        loss2 = criterion(outputs2,labels2)
        loss3 = criterion(outputs3,labels3)
        running_loss += loss1+loss2+loss3
        running_acc += (outputs1.argmax(1)==labels1).float().mean()
        running_acc += (outputs2.argmax(1)==labels2).float().mean()
        running_acc += (outputs3.argmax(1)==labels3).float().mean()
        (loss1+loss2+loss3).backward()
        optimizer.step()
    #scheduler.step()
    losses.append(running_loss/len(train_loader))
    accs.append(running_acc/(len(train_loader)*3))
    print('acc : {:.2f}%'.format(running_acc/(len(train_loader)*3)))
    print('loss : {:.4f}'.format(running_loss/len(train_loader)))
torch.save(model.state_dict(), 'resnet34_50epochs_saved_weights.pth')

In [ ]:
# plot 
fig,ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(losses)
ax[0].set_title('loss')
ax[1].plot(accs)
ax[1].set_title('acc')

In [ ]:
# ... now finally efficientnet

In [ ]:
HEIGHT = 137
WIDTH = 236
FACTOR = 0.70
HEIGHT_NEW = int(HEIGHT * FACTOR)
WIDTH_NEW = int(WIDTH * FACTOR)
CHANNELS = 3
BATCH_SIZE = 16

In [ ]:
# Image Prep for EfficientNet
def resize_image(img, WIDTH_NEW, HEIGHT_NEW):
    # Invert
    img = 255 - img

    # Normalize
    img = (img * (255.0 / img.max())).astype(np.uint8)

    # Reshape
    img = img.reshape(HEIGHT, WIDTH)
    image_resized = cv2.resize(img, (WIDTH_NEW, HEIGHT_NEW), interpolation = cv2.INTER_AREA)

    return image_resized.reshape(-1)   

In [ ]:
# Generalized mean pool - GeM
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

In [ ]:
# Create Model
def create_model(input_shape):
    # Input Layer
    input = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = efn.EfficientNetB3(weights = None, include_top = False, input_tensor = input, pooling = None, classes = None)
    
    # UnFreeze all layers
    for layer in x_model.layers:
        layer.trainable = True
    
    # GeM
    lambda_layer = Lambda(generalized_mean_pool_2d)
    lambda_layer.trainable_weights.extend([gm_exp])
    x = lambda_layer(x_model.output)
    
    # multi output
    grapheme_root = Dense(168, activation = 'softmax', name = 'root')(x)
    vowel_diacritic = Dense(11, activation = 'softmax', name = 'vowel')(x)
    consonant_diacritic = Dense(7, activation = 'softmax', name = 'consonant')(x)

    # model
    model = Model(inputs = x_model.input, outputs = [grapheme_root, vowel_diacritic, consonant_diacritic])

    return model

In [ ]:
# Create Model
model1 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model2 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model3 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model4 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model5 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))

In [ ]:
# Load Model Weights : these are pretrained model weights that save us a lot of time
model1.load_weights('../input/kerasefficientnetb3/Train1_model_59.h5') 
model2.load_weights('../input/kerasefficientnetb3/Train1_model_64.h5') 
model3.load_weights('../input/kerasefficientnetb3/Train1_model_68.h5') 
model4.load_weights('../input/kerasefficientnetb3/Train1_model_57.h5') 
model5.load_weights('../input/kerasefficientnetb3/Train1_model_70.h5') 

In [ ]:
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']

# Create Predictions
row_ids, targets = [], []

# Loop through Test Parquet files (X)
for i in range(0, 4):
    # Test Files Placeholder
    test_files = []

    # Read Parquet file
    df = pd.read_parquet(os.path.join(DIR, 'test_image_data_'+str(i)+'.parquet'))
    # Get Image Id values
    image_ids = df['image_id'].values 
    # Drop Image_id column
    df = df.drop(['image_id'], axis = 1)

    # Loop over rows in Dataframe and generate images 
    X = []
    for image_id, index in zip(image_ids, range(df.shape[0])):
        test_files.append(image_id)
        X.append(resize_image(df.loc[df.index[index]].values, WIDTH_NEW, HEIGHT_NEW))

    # Data_Generator
    data_generator_test = TestDataGenerator(X, batch_size = BATCH_SIZE, img_size = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
        
    # Predict with all 3 models
    preds1 = model1.predict_generator(data_generator_test, verbose = 1)
    preds2 = model2.predict_generator(data_generator_test, verbose = 1)
    preds3 = model3.predict_generator(data_generator_test, verbose = 1)
    preds4 = model4.predict_generator(data_generator_test, verbose = 1)
    preds5 = model5.predict_generator(data_generator_test, verbose = 1)
    
    # Loop over Preds    
    for i, image_id in zip(range(len(test_files)), test_files):
        
        for subi, col in zip(range(len(preds1)), tgt_cols):
            sub_preds1 = preds1[subi]
            sub_preds2 = preds2[subi]
            sub_preds3 = preds3[subi]
            sub_preds4 = preds4[subi]
            sub_preds5 = preds5[subi]

            # Set Prediction with average of 5 predictions
            row_ids.append(str(image_id)+'_'+col)
            sub_pred_value = np.argmax((sub_preds1[i] + sub_preds2[i] + sub_preds3[i] + sub_preds4[i] + sub_preds5[i]) / 5)
            targets.append(sub_pred_value)